In [141]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session\
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error

/kaggle/input/round-2-nexus-recruitment/sample_submission.csv
/kaggle/input/round-2-nexus-recruitment/train.csv
/kaggle/input/round-2-nexus-recruitment/test.csv


In [142]:
train_df = pd.read_csv('/kaggle/input/round-2-nexus-recruitment/train.csv')
train_df

,id,Hardness,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,0,2.5,76.0,13.053996,44.500000,5.300000,102.827560,9.273855,2.274000,1.907200,1.180000,0.457149,5.54333
1,1,5.5,132.0,8.329324,11.600000,4.333333,22.436766,10.599883,2.575833,1.796667,0.973333,0.481969,0.45758
2,2,6.8,92.0,6.283996,9.200000,4.800000,18.608172,11.144460,2.698000,1.732000,0.920000,0.496167,1.03881
3,3,2.3,45.0,7.441332,13.000000,5.500000,29.525338,10.347000,2.904000,1.720000,0.945000,0.481478,1.86481
4,4,6.5,126.0,13.747992,13.900000,4.650000,29.680679,11.759620,2.649500,1.720000,0.958000,0.486265,2.48360
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2.8,459.0,37.007299,30.500000,5.271667,67.939400,10.789170,2.279000,1.907200,0.907200,0.466938,0.36822
14996,14996,5.5,220.0,14.055328,11.333333,4.666667,24.789618,11.252967,2.831667,1.734167,0.933333,0.481072,0.82082
14997,14997,3.8,63.0,13.385996,12.500000,4.800000,26.385218,11.381680,2.866000,1.695000,0.786000,0.477854,1.39322
14998,14998,6.0,100.0,8.836996,10.000000,4.800000,20.392033,11.166820,2.752000,1.754000,0.908000,0.491604,1.04896


In [143]:
test_df = pd.read_csv('/kaggle/input/round-2-nexus-recruitment/test.csv')
test_df

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,15000,86.0,11.354575,14.333333,4.000000,30.550687,11.948200,2.953333,1.725000,0.966667,0.467532,1.35155
1,15001,100.0,5.203996,10.000000,4.800000,20.298893,12.040830,2.790000,1.631000,0.910000,0.493032,0.75893
2,15002,79.0,13.551332,19.000000,4.000000,39.568056,9.676900,2.140000,1.880000,1.130000,0.477665,2.31200
3,15003,92.0,6.697992,9.200000,4.800000,18.439538,11.798860,2.805000,1.656000,0.848000,0.499514,0.77680
4,15004,144.0,13.512992,14.400000,4.400000,31.937418,11.331820,2.824000,1.752000,0.840000,0.499404,1.25093
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,94.0,10.584328,15.666667,4.333333,35.457915,11.526700,2.810000,1.746667,0.893333,0.478880,1.51422
9996,24996,68.0,4.243996,13.600000,4.800000,29.525338,10.347000,2.727000,1.767000,0.910000,0.489272,2.45933
9997,24997,122.0,14.220000,20.666667,4.666667,46.443748,9.540800,2.330000,1.880000,1.106667,0.487833,1.39322
9998,24998,0.0,0.679221,0.000000,0.000000,0.000000,11.654067,0.000000,0.000000,0.000000,0.499507,0.66980


In [144]:
X = np.zeros(15000)
X = train_df.iloc[:, 2:13]
X

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,76.0,13.053996,44.500000,5.300000,102.827560,9.273855,2.274000,1.907200,1.180000,0.457149,5.54333
1,132.0,8.329324,11.600000,4.333333,22.436766,10.599883,2.575833,1.796667,0.973333,0.481969,0.45758
2,92.0,6.283996,9.200000,4.800000,18.608172,11.144460,2.698000,1.732000,0.920000,0.496167,1.03881
3,45.0,7.441332,13.000000,5.500000,29.525338,10.347000,2.904000,1.720000,0.945000,0.481478,1.86481
4,126.0,13.747992,13.900000,4.650000,29.680679,11.759620,2.649500,1.720000,0.958000,0.486265,2.48360
...,...,...,...,...,...,...,...,...,...,...,...
14995,459.0,37.007299,30.500000,5.271667,67.939400,10.789170,2.279000,1.907200,0.907200,0.466938,0.36822
14996,220.0,14.055328,11.333333,4.666667,24.789618,11.252967,2.831667,1.734167,0.933333,0.481072,0.82082
14997,63.0,13.385996,12.500000,4.800000,26.385218,11.381680,2.866000,1.695000,0.786000,0.477854,1.39322
14998,100.0,8.836996,10.000000,4.800000,20.392033,11.166820,2.752000,1.754000,0.908000,0.491604,1.04896


In [145]:
y = np.zeros(15000)
y = train_df.iloc[:, 1]
y

0        2.5
1        5.5
2        6.8
3        2.3
4        6.5
        ... 
14995    2.8
14996    5.5
14997    3.8
14998    6.0
14999    5.8
Name: Hardness, Length: 15000, dtype: float64

In [146]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [147]:
model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    min_child_weight=1,
    gamma=0.0,
    subsample=0.8,
    colsample_bytree=0.6,
    reg_alpha=0.3,
    reg_lambda=0.8,
    early_stopping_rounds=10,
    eval_metric="mae"         
)


model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)], 
    verbose=True
)

[0]	validation_0-mae:1.50519
[1]	validation_0-mae:1.47696
[2]	validation_0-mae:1.45180
[3]	validation_0-mae:1.42761
[4]	validation_0-mae:1.40263
[5]	validation_0-mae:1.37865
[6]	validation_0-mae:1.35751
[7]	validation_0-mae:1.33626
[8]	validation_0-mae:1.31676
[9]	validation_0-mae:1.29883
[10]	validation_0-mae:1.28196
[11]	validation_0-mae:1.26704
[12]	validation_0-mae:1.25092
[13]	validation_0-mae:1.23712
[14]	validation_0-mae:1.22294
[15]	validation_0-mae:1.21097
[16]	validation_0-mae:1.19791
[17]	validation_0-mae:1.18716
[18]	validation_0-mae:1.17618
[19]	validation_0-mae:1.16607
[20]	validation_0-mae:1.15676
[21]	validation_0-mae:1.14721
[22]	validation_0-mae:1.13867
[23]	validation_0-mae:1.13096
[24]	validation_0-mae:1.12297
[25]	validation_0-mae:1.11587
[26]	validation_0-mae:1.10902
[27]	validation_0-mae:1.10220
[28]	validation_0-mae:1.09624
[29]	validation_0-mae:1.08989
[30]	validation_0-mae:1.08382
[31]	validation_0-mae:1.07785
[32]	validation_0-mae:1.07232
[33]	validation_0-ma

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=10,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=0.0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [149]:
y_pred_val = model.predict(X_val)
print(y_pred_val)

[4.8553915 5.739536  5.429089  ... 3.465996  5.406543  4.7884727]


In [150]:
medae = median_absolute_error(y_val, y_pred_val)
# Output the result
print(f'Median Absolute Error: {medae}')

Median Absolute Error: 0.7317438125610352


In [152]:
X_test = np.zeros(15000)
X_test = test_df.iloc[:, 1:13]
X_test

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,86.0,11.354575,14.333333,4.000000,30.550687,11.948200,2.953333,1.725000,0.966667,0.467532,1.35155
1,100.0,5.203996,10.000000,4.800000,20.298893,12.040830,2.790000,1.631000,0.910000,0.493032,0.75893
2,79.0,13.551332,19.000000,4.000000,39.568056,9.676900,2.140000,1.880000,1.130000,0.477665,2.31200
3,92.0,6.697992,9.200000,4.800000,18.439538,11.798860,2.805000,1.656000,0.848000,0.499514,0.77680
4,144.0,13.512992,14.400000,4.400000,31.937418,11.331820,2.824000,1.752000,0.840000,0.499404,1.25093
...,...,...,...,...,...,...,...,...,...,...,...
9995,94.0,10.584328,15.666667,4.333333,35.457915,11.526700,2.810000,1.746667,0.893333,0.478880,1.51422
9996,68.0,4.243996,13.600000,4.800000,29.525338,10.347000,2.727000,1.767000,0.910000,0.489272,2.45933
9997,122.0,14.220000,20.666667,4.666667,46.443748,9.540800,2.330000,1.880000,1.106667,0.487833,1.39322
9998,0.0,0.679221,0.000000,0.000000,0.000000,11.654067,0.000000,0.000000,0.000000,0.499507,0.66980


In [153]:
y_predicted_test = model.predict(X_test)
y_predicted_test

array([4.577409 , 4.678792 , 4.3472514, ..., 5.1464863, 3.6730604,
       5.584801 ], dtype=float32)

In [154]:
id_df = test_df.iloc[:, 0]
id_df

0       15000
1       15001
2       15002
3       15003
4       15004
        ...  
9995    24995
9996    24996
9997    24997
9998    24998
9999    24999
Name: id, Length: 10000, dtype: int64

In [155]:
final_df = pd.DataFrame(id_df)
final_df['Hardness'] = y_predicted_test
final_df


,id,Hardness
0,15000,4.577409
1,15001,4.678792
2,15002,4.347251
3,15003,4.053985
4,15004,5.305066
...,...,...
9995,24995,5.528720
9996,24996,5.027150
9997,24997,5.146486
9998,24998,3.673060


In [156]:
final_df.to_csv('hardness_prediction9.csv', index=False)